In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import time

pd.options.display.expand_frame_repr = False # запрещаем перенос строк

In [2]:
def get_date_title_link_text_likes_from_habr(res):
  '''
  Функция получает ответ от сайта habr.ru, находит в нем дату,  заголовок и ссылку поста,
  затем делает запрос по ссылке и находит в ней текст.
  Возвращает полученные данные в виде df.
  '''
  soup = bs(res.text)
  posts = soup.find_all('div', class_='tm-article-snippet tm-article-snippet')
  habr = pd.DataFrame()
  for post in posts:
    date_post_ = post.find('time').text
    title_ = post.find('a', 'tm-article-snippet__title-link').text
    link_ = post.find('a', 'tm-article-snippet__title-link').get('href')
    time.sleep(0.3)
    res_to_link = requests.get(f'https://habr.com{link_}')
    soup_to_link = bs(res_to_link.text)
    try:
      text_ = soup_to_link.find('div', xmlns="http://www.w3.org/1999/xhtml").text.strip()
    except:
      text_ = 'Не смог распарсить текст'
    likes_ = soup_to_link.find('span', class_="tm-votes-meter__value tm-votes-meter__value tm-votes-meter__value_positive tm-votes-meter__value_appearance-article tm-votes-meter__value_rating")
    row = {'date': date_post_, 'title': title_, 'link': link_, 'text': text_, 'likes': likes_}
    habr = pd.concat([habr, pd.DataFrame([row])])
  return habr


In [3]:
def get_data_from_habr(search_list, pages=1):
  '''
  Функция получает на вход список фраз и количество страниц поиска для запроса с сайта habr.ru,
  отправляет запрос на сайт, передает ответ каждой страницы функции get_date_title_link_text_from_habr,
  получает df с датой, заголовком, ссылкой и текстом поста и возвращает его
  '''
  for search_string in search_list:
    result = pd.DataFrame()
    for page in range(1, pages + 1):
      URL = f'https://habr.com/ru/search/page{page}/'
      params = {'q': search_string, 'target_type': 'posts', 'order': 'relevance'}
      res = requests.get(URL, params=params)
      time.sleep(0.3)
      result = pd.concat([result, get_date_title_link_text_likes_from_habr(res)])
    print(f'Поиск фразы "{search_string}" на {pages} странице(страницах) habr.ru:\n', result.reset_index(drop=True))
  

In [4]:
get_data_from_habr(search_list=['python', 'аналитика данных'], pages=3)

Поиск фразы "python" на 3 странице(страницах) habr.ru:
                           date                                              title                                         link                                               text   likes
0         20  января   в 18:37  Курс «Python для инженеров». Старт 3 потока 31...       /ru/company/southbridge/news/t/646825/  Курс нацелен дать максимальную пользу, поэтому...   [+10]
1    13  декабря  2021 в 09:00  Жаждущим автоматизации: открытый урок «ChatOps...       /ru/company/southbridge/news/t/595093/  21 декабря Слёрм проведёт открытый урок «ChatO...    [+9]
2     21  апреля  2020 в 18:35  Вышел Python 2.7.18, последний релиз ветки Pyt...                           /ru/news/t/498364/  20 апреля 2020 года, спустя почти десять лет п...   [+19]
3        6  июля  2021 в 13:29  Python Community Meetup 8/07: видео и материал...    /ru/company/raiffeisenbank/news/t/566370/  Первый открытый онлайн-митап сообщества, для к...    [+3]
4         13  